In [135]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse
import pandas as pd
import math
import matplotlib.pyplot as plt

### Load Data

In [136]:
### change `DATA_DIR` to the location where movielens-20m dataset sits
DATA_DIR = 'C:/Users/SEO/PycharmProjects/testdigo/datasets/ML_20m'

In [185]:
min_user = 100
min_product = 0

In [186]:
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)

In [187]:
raw_data = raw_data[raw_data['rating'] > 3.5]

In [188]:
raw_data.head()

,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703


In [189]:
raw_data.shape

(9995410, 4)

In [190]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count

In [191]:
def filter_triplets(tp, min_uc=0, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
         #기존 data를 min_sc 해당되는 movield만 filtering       
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        #기존 data를 min_uc이상 해당되는 userld만 filtering
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, itemcount

In [192]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_user, min_product)

In [193]:
unique_uid = user_activity.index
idx_permperm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_permperm]
print(unique_uid)

Int64Index([ 49229, 127340, 111640,  82866, 130533,  49239, 121076,  85007,
            118034,  82658,
            ...
             99671,  88261, 120435,  14319,  79965,  59539, 127836,  56214,
             45962,  33842],
           dtype='int64', name='userId', length=28646)


In [194]:
unique_sid = pd.unique(raw_data['movieId'])

In [195]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [196]:
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['userId']))
    sid = list(map(lambda x: show2id[x], tp['movieId']))
    print(uid[0])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [197]:
pro_dir = "./"

if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid_m_test.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

In [198]:
def getSparsity(data):
    usercount = get_count(data,'userId')
    itemcount = get_count(data, 'movieId')
    sparsity = 1. * data.shape[0] / (usercount.shape[0] * len(unique_sid))
    print("After filtering, there are %d watching events from %d users and %d products (sparsity: %.3f%%)" % 
      (data.shape[0], usercount.shape[0], len(unique_sid), sparsity * 100))   

In [199]:
getSparsity(raw_data)

After filtering, there are 6141416 watching events from 28646 users and 20210 products (sparsity: 1.061%)


In [200]:
train_data = numerize(raw_data)
train_data.to_csv(os.path.join(pro_dir, 'train_test.csv'), index=False)

26586
